In [1]:
import pandas as pd

data = pd.read_csv('games.csv', index_col=0)

In [2]:
# Cleaning / getting data ready for machine learning
data['venue'] = data['venue'].map({'Home' : 1, 'Away' : 0})     # convert venue to 1's and 0's

In [3]:
data

,Date,Time,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result
0,2021-12-12,7:00 PM,Anaheim Ducks,St. Louis Blues,0,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,1
1,2023-01-28,10:30 PM,Anaheim Ducks,Arizona Coyotes,1,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,1
2,2022-01-14,8:00 PM,Anaheim Ducks,Minnesota Wild,0,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0
3,2024-11-05,10:00 PM,Anaheim Ducks,Vancouver Canucks,1,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0
4,2022-10-12,10:00 PM,Anaheim Ducks,Seattle Kraken,1,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,8:00 PM,Winnipeg Jets,Tampa Bay Lightning,1,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1
10492,2025-03-07,7:00 PM,Winnipeg Jets,New Jersey Devils,0,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1
10493,2023-02-20,7:00 PM,Winnipeg Jets,New York Rangers,0,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1
10494,2025-03-09,5:00 PM,Winnipeg Jets,Carolina Hurricanes,0,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,0


In [4]:
data['Opponent'] = data['Opponent'].astype('category').cat.codes      # opponent is now represented by a number
data

,Date,Time,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result
0,2021-12-12,7:00 PM,Anaheim Ducks,25,0,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,1
1,2023-01-28,10:30 PM,Anaheim Ducks,1,1,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,1
2,2022-01-14,8:00 PM,Anaheim Ducks,14,0,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0
3,2024-11-05,10:00 PM,Anaheim Ducks,29,1,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0
4,2022-10-12,10:00 PM,Anaheim Ducks,24,1,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,8:00 PM,Winnipeg Jets,26,1,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1
10492,2025-03-07,7:00 PM,Winnipeg Jets,17,0,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1
10493,2023-02-20,7:00 PM,Winnipeg Jets,19,0,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1
10494,2025-03-09,5:00 PM,Winnipeg Jets,5,0,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,0


In [5]:
games = data.drop(columns=['Time'], inplace=False)
games

,Date,Team,Opponent,venue,Att.,G,GA,S,S%,SV%,PIM,SA,SA%,Opponent SV%,Opponent PIM,Result
0,2021-12-12,Anaheim Ducks,25,0,17010.0,3,2,39,7.7,0.920,2,25,8.0,0.923,2,1
1,2023-01-28,Anaheim Ducks,1,1,16126.0,2,1,45,4.4,0.971,20,34,2.9,0.956,10,1
2,2022-01-14,Anaheim Ducks,14,0,18300.0,3,7,42,7.1,0.793,5,42,16.7,0.929,7,0
3,2024-11-05,Anaheim Ducks,29,1,13538.0,1,5,22,4.5,0.865,10,37,13.5,0.955,8,0
4,2022-10-12,Anaheim Ducks,24,1,17530.0,5,4,27,18.5,0.917,15,48,8.3,0.815,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10491,2024-01-02,Winnipeg Jets,26,1,14157.0,4,2,28,14.3,0.941,6,34,5.9,0.889,6,1
10492,2025-03-07,Winnipeg Jets,17,0,16088.0,6,1,35,17.1,0.957,4,23,4.3,0.829,4,1
10493,2023-02-20,Winnipeg Jets,19,0,18006.0,4,1,21,19.0,0.980,13,51,2.0,0.810,7,1
10494,2025-03-09,Winnipeg Jets,5,0,18700.0,2,4,22,9.1,0.885,24,27,14.8,0.909,8,0


In [6]:
training = games[games['Date'] < '2024-04-19']  # Training using 2021-2024 data
testing = games[games['Date'] > '2024-04-19']   # Testing on most recent season (2024-2025)

import xgboost as xgb

In [7]:
# Dropping 'Date' column because not needed anymore
training = training.drop(columns=['Date'])
testing = testing.drop(columns=['Date'])


In [12]:
# Getting training setup
x_train = training.drop(columns=['Result'])
y_train = training['Result']

# Getting testing setup
x_test = testing.drop(columns=['Result'])
y_test = testing['Result']

In [11]:
from xgboost import XGBClassifier

model = XGBClassifier(random_state=10, verbosity=2)

In [16]:
# Defining search space for GridSearchCV
search_grid = {
    'n_estimators' : [50, 100, 200, 500],
    'max_depth' : [3, 6, 9],
    'learning_rate' : [0.001, 0.01, 0.1],
    'reg_alpha': [0, 0.1, 1, 5, 10],
    'reg_lambda': [0.1, 1, 5, 10, 20],

}